In [1]:
import json
import folium
import requests
import mimetypes
import http.client
import pandas as pd
from folium.plugins import HeatMap
from pandas.io.json import json_normalize

In [2]:
conn = http.client.HTTPSConnection('api.covid19api.com')
payload =''
headers={}
conn.request('GET', '/summary', payload, headers)
res=conn.getresponse()
data=res.read().decode('UTF-8')

In [3]:
covid1=json.loads(data)


In [4]:
from pandas.io.json import json_normalize
pd.json_normalize(covid1["Countries"], sep=",").drop(columns=["ID"])

,Country,CountryCode,Slug,NewConfirmed,TotalConfirmed,NewDeaths,TotalDeaths,NewRecovered,TotalRecovered,Date
0,Afghanistan,AF,afghanistan,76,54559,3,2373,31,46943,2021-01-25T00:43:52.561Z
1,Albania,AL,albania,786,71441,7,1310,489,43384,2021-01-25T00:43:52.561Z
2,Algeria,DZ,algeria,245,105369,5,2861,203,71755,2021-01-25T00:43:52.561Z
3,Andorra,AD,andorra,83,9499,3,96,51,8588,2021-01-25T00:43:52.561Z
4,Angola,AO,angola,98,19367,5,457,26,17249,2021-01-25T00:43:52.561Z
...,...,...,...,...,...,...,...,...,...,...
184,Venezuela (Bolivarian Republic),VE,venezuela,450,123245,6,1142,501,115426,2021-01-25T00:43:52.561Z
185,Viet Nam,VN,vietnam,0,1548,0,35,0,1411,2021-01-25T00:43:52.561Z
186,Yemen,YE,yemen,0,2118,1,615,0,1423,2021-01-25T00:43:52.561Z
187,Zambia,ZM,zambia,1259,44592,17,627,1626,34293,2021-01-25T00:43:52.561Z


In [5]:
df=pd.DataFrame(covid1['Countries']).drop(columns=['ID'])



In [6]:
covid2 = df.drop(columns=['Slug', 'CountryCode', 'Date', 'Premium'])


In [7]:
m = folium.Map(tiles = "Stamen Terrain", min_zoom=1.5)


In [8]:
url= 'http://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
country_shapes = f'{url}/world-countries.json'

In [9]:
folium.Choropleth(
    geo_data=country_shapes,
    min_zoom=2,
    data =covid2,
    name='Covid-19',
    columns = ['Country', 'TotalConfirmed'],
    key_on = 'feature.properties.name',
    fill_color = 'OrRd',
    nan_fill_color = 'black',
    legend_name = 'Total Confirmed Covid Cases'
).add_to(m)
m

In [10]:
covid2.update(covid2['TotalConfirmed'].map('Total Confirmed:{}'.format))
covid2.update(covid2['TotalRecovered'].map('Total Recovered:{}'.format))

In [11]:
coordinates = pd.read_csv('https://raw.githubusercontent.com/VinitaSilaparasetty/covid-map/master/country-coordinates-world.csv')


In [12]:
covid_final = pd.merge(covid2, coordinates, on='Country');

In [13]:
def DotPlot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                       radius=5,
                       weight=2, 
                       popup=[point.Country, point.TotalConfirmed, point.TotalRecovered], 
                       fill_color='#000000').add_to(m)

In [14]:
covid_final.apply(DotPlot, axis=1)
m.fit_bounds(m.get_bounds())
m

In [15]:
m1=folium.Map(tiles ='StamenToner', min_zoom=2)


In [16]:
deaths = covid_final['TotalDeaths'].astype(float)

In [17]:
lat = covid_final['latitude'].astype(float)
lon = covid_final['longitude'].astype(float)

In [18]:
m1.add_child(HeatMap(zip(lat, lon, deaths), radius=0))


In [19]:
def DotPlot(point):
    folium.CircleMarker(location=[point.latitude, point.longitude],
                       radius=5,
                       weight=2, 
                       popup=[point.Country, point.TotalDeaths], 
                       fill_color='#000000').add_to(m1)

In [20]:
covid_final.apply(DotPlot, axis=1)
m1.fit_bounds(m1.get_bounds())
m1